In [48]:
from symbol import if_stmt

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.ndimage import median_filter, rotate, shift
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score
from torch.xpu import device

In [49]:
df = pd.read_pickle("/home/unsi/다운로드/LSWMD.pkl")  #/home/unsi/다운로드/LSWMD.pkl /Users/unsi/Downloads/LSWMD.pkl
#ㅇㅇ 

In [50]:
df.iloc[35]['failureType']


array([], shape=(0, 0), dtype=float64)

In [51]:
# 빈 리스트를 포함하는 행만 필터링하기
passed = df[df['failureType'].apply(lambda x: x.shape == (0, 0) or x[0][0] == 'none')]
passed['failureType'] = 8
passed

/tmp/ipykernel_15097/3097371801.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  passed['failureType'] = 8


,waferMap,dieSize,lotName,waferIndex,trianTestLabel,failureType
0,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1683.0,lot1,1.0,[[Training]],8
1,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1683.0,lot1,2.0,[[Training]],8
2,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1683.0,lot1,3.0,[[Training]],8
3,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1683.0,lot1,4.0,[[Training]],8
4,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1683.0,lot1,5.0,[[Training]],8
...,...,...,...,...,...,...
811442,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 1, 1, 1,...",600.0,lot47542,13.0,[[Test]],8
811445,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,...",600.0,lot47542,16.0,[[Test]],8
811449,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,...",600.0,lot47542,20.0,[[Test]],8
811455,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,...",600.0,lot47543,1.0,[],8


In [52]:
# Filtering rows based on a nested condition in the 'failureType' column
faild = df[df['failureType'].apply(lambda x: len(x) > 0 and len(x[0]) > 0 and x[0][0] != 'none')]
faild

,waferMap,dieSize,lotName,waferIndex,trianTestLabel,failureType
19,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1683.0,lot1,20.0,[[Training]],[[Loc]]
36,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",2460.0,lot2,12.0,[[Training]],[[Edge-Loc]]
37,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",2460.0,lot2,13.0,[[Training]],[[Edge-Loc]]
38,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",2460.0,lot2,14.0,[[Training]],[[Edge-Loc]]
39,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",2460.0,lot2,15.0,[[Training]],[[Edge-Loc]]
...,...,...,...,...,...,...
811450,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 1, 2,...",600.0,lot47542,21.0,[[Test]],[[Edge-Loc]]
811451,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,...",600.0,lot47542,22.0,[[Test]],[[Edge-Loc]]
811452,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 1, 1,...",600.0,lot47542,23.0,[[Test]],[[Edge-Ring]]
811453,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 1, 1,...",600.0,lot47542,24.0,[[Test]],[[Edge-Loc]]


In [53]:
faild.columns

Index(['waferMap', 'dieSize', 'lotName', 'waferIndex', 'trianTestLabel',
       'failureType'],
      dtype='object')

In [54]:


# failureType을 숫자로 변환
class2idx = {
    'Loc': 0,
    'Edge-Loc': 1,
    'Center': 2,
    'Edge-Ring': 3,
    'Scratch': 4,
    'Random': 5,
    'Near-full': 6,
    'Donut': 7,
    'pass': 8
}

# map 함수를 사용하여 변환
faild['failureType'] = faild['failureType'].apply(lambda x: class2idx.get(x[0][0], -1))

# 결과 확인
faild


/tmp/ipykernel_15097/1877288277.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  faild['failureType'] = faild['failureType'].apply(lambda x: class2idx.get(x[0][0], -1))


,waferMap,dieSize,lotName,waferIndex,trianTestLabel,failureType
19,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1683.0,lot1,20.0,[[Training]],0
36,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",2460.0,lot2,12.0,[[Training]],1
37,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",2460.0,lot2,13.0,[[Training]],1
38,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",2460.0,lot2,14.0,[[Training]],1
39,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",2460.0,lot2,15.0,[[Training]],1
...,...,...,...,...,...,...
811450,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 1, 2,...",600.0,lot47542,21.0,[[Test]],1
811451,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,...",600.0,lot47542,22.0,[[Test]],1
811452,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 1, 1,...",600.0,lot47542,23.0,[[Test]],3
811453,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 1, 1,...",600.0,lot47542,24.0,[[Test]],1


In [55]:
import numpy as np
import pandas as pd


def augment_wafer_map(wafer_map):
    # 좌우 뒤집기
    flipped_lr = np.fliplr(wafer_map)

    # 상하 뒤집기
    flipped_ud = np.flipud(wafer_map)

    # 90도 회전 (시계 방향)
    rotated_90 = np.rot90(wafer_map, k=-1)

    # 180도 회전
    rotated_180 = np.rot90(wafer_map, k=2)

    # 270도 회전 (반시계 방향)
    rotated_270 = np.rot90(wafer_map, k=1)

    return [wafer_map, flipped_lr, flipped_ud, rotated_90, rotated_180, rotated_270]


def augment_dataframe(df):
    augmented_data = []

    for _, row in df.iterrows():
        wafer_map = row['waferMap']
        die_size = row['dieSize']
        lot_name = row['lotName']
        wafer_index = row['waferIndex']
        train_test_label = row['trianTestLabel']
        failure_type = row['failureType']

        # 각 증강된 waferMap과 원본을 리스트로 가져옴
        augmented_wafer_maps = augment_wafer_map(wafer_map)

        # 각 증강된 waferMap에 대해 새로운 행 추가
        for augmented_map in augmented_wafer_maps:
            augmented_data.append([
                augmented_map,
                die_size,
                lot_name,
                wafer_index,
                train_test_label,
                failure_type
            ])

    # 증강된 데이터로 새로운 DataFrame 생성 (기존 컬럼과 동일한 순서와 이름으로 생성)
    augmented_df = pd.DataFrame(augmented_data,
                                columns=['waferMap', 'dieSize', 'lotName', 'waferIndex', 'trianTestLabel',
                                         'failureType'])

    # 원본 df와 증강된 df를 concat하여 반환
    combined_df = pd.concat([df, augmented_df], ignore_index=True)
    return combined_df


augmented_faild = augment_dataframe(faild)

# 결과 확인
print("Original DataFrame shape:", faild.shape)
print("Augmented DataFrame shape:", augmented_faild.shape)


Original DataFrame shape: (25519, 6)
Augmented DataFrame shape: (178633, 6)


In [56]:
augmented_faild['failureType'].value_counts()

failureType
3    67760
1    36323
2    30058
0    25151
4     8351
5     6062
7     3885
6     1043
Name: count, dtype: int64

In [57]:
passed = passed.sample(n=100000, random_state=42).reset_index(drop=True)

In [58]:
passed

,waferMap,dieSize,lotName,waferIndex,trianTestLabel,failureType
0,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1513.0,lot30592,18.0,[],8
1,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2,...",980.0,lot11868,17.0,[],8
2,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1421.0,lot44781,7.0,[[Test]],8
3,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 1,...",710.0,lot35710,23.0,[],8
4,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2,...",895.0,lot47261,9.0,[[Test]],8
...,...,...,...,...,...,...
99995,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",5532.0,lot20174,4.0,[],8
99996,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 1,...",710.0,lot38319,13.0,[],8
99997,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 1, 2,...",648.0,lot15275,11.0,[],8
99998,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",2393.0,lot22590,20.0,[],8


In [59]:
df = pd.concat([passed, augmented_faild], ignore_index=True)

In [60]:
df

,waferMap,dieSize,lotName,waferIndex,trianTestLabel,failureType
0,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1513.0,lot30592,18.0,[],8
1,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2,...",980.0,lot11868,17.0,[],8
2,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1421.0,lot44781,7.0,[[Test]],8
3,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 1,...",710.0,lot35710,23.0,[],8
4,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2,...",895.0,lot47261,9.0,[[Test]],8
...,...,...,...,...,...,...
278628,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,...",600.0,lot47542,25.0,[[Test]],3
278629,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2,...",600.0,lot47542,25.0,[[Test]],3
278630,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2,...",600.0,lot47542,25.0,[[Test]],3
278631,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2,...",600.0,lot47542,25.0,[[Test]],3


In [62]:
def apply_median_filter(wafer_map, size=3):
    return median_filter(wafer_map, size=size)

# DataFrame에 있는 모든 WaferMap에 미디언 필터 적용
df['waferMap'] = df['waferMap'].apply(lambda x: apply_median_filter(x, size=3))


In [63]:
df.dieSize.value_counts()

dieSize
515.0     21949
1075.0    14407
710.0     10718
2126.0    10634
3036.0    10348
          ...  
3021.0        1
3079.0        1
1771.0        1
1814.0        1
2833.0        1
Name: count, Length: 951, dtype: int64

In [64]:
df.failureType.value_counts()

failureType
8    100000
3     67760
1     36323
2     30058
0     25151
4      8351
5      6062
7      3885
6      1043
Name: count, dtype: int64

In [65]:
import numpy as np
import pandas as pd
from joblib import Parallel, delayed
from tqdm import tqdm


def extract_statistics(data):
    # 데이터가 0, 1, 2만 포함되어 있다고 가정하고 로그 변환 수행
    data_log = np.log1p(data)  # log(1 + x)로 변환하여 0 값 문제 해결

    # 다이의 크기
    rows, cols = data_log.shape

    # 가로와 세로를 3개로 나누어 9개의 구간 생성
    row_splits = np.array_split(np.arange(rows), 3)
    col_splits = np.array_split(np.arange(cols), 3)

    statistics = []

    for row_idx in row_splits:
        for col_idx in col_splits:
            # 각 구간의 데이터를 추출
            section = data_log[np.ix_(row_idx, col_idx)]

            # 통계 데이터 계산
            stats = [
                np.mean(section),
                np.std(section),
                np.var(section),
                np.min(section),
                np.max(section),
                np.sum(section),
                np.median(section),
                np.ptp(section)  # 최대값 - 최소값
            ]
            statistics.extend(stats)

    # 리스트를 ndarray로 변환하여 반환
    return np.array(statistics)


def process_row(row):
    wafer_map = row['waferMap']
    failure_type = row['failureType']
    features = extract_statistics(wafer_map)
    return features, failure_type


# 병렬 처리 설정 (CPU 코어 수에 맞게 병렬 처리 수행)
num_cores = -1  # 모든 CPU 코어 사용
results = Parallel(n_jobs=num_cores)(
    delayed(process_row)(row) for _, row in tqdm(df.iterrows(), total=len(df))
)

# features_array와 labels_array 생성
features_array = np.array([result[0] for result in results])
labels_array = np.array([result[1] for result in results])

# 결과 확인
print("Features array shape:", features_array.shape)  # 통계 데이터의 형태 확인
print("Labels array shape:", labels_array.shape)  # 라벨 데이터의 형태 확인
print(features_array[0])  # 첫 번째 행의 통계 데이터 출력
print(labels_array[0])  # 첫 번째 행의 라벨 출력


100%|██████████| 278633/278633 [00:30<00:00, 9285.58it/s] 


Features array shape: (278633, 72)
Labels array shape: (278633,)
[3.7329e-01 3.4570e-01 1.1951e-01 0.0000e+00 6.9336e-01 8.2500e+01
 6.9336e-01 6.9336e-01 6.8066e-01 9.2468e-02 8.5449e-03 0.0000e+00
 6.9336e-01 1.5050e+02 6.9336e-01 6.9336e-01 3.8892e-01 3.4399e-01
 1.1841e-01 0.0000e+00 6.9336e-01 8.6000e+01 6.9336e-01 6.9336e-01
 6.8750e-01 9.2041e-02 8.4763e-03 0.0000e+00 1.0986e+00 1.4300e+02
 6.9336e-01 1.0986e+00 6.9336e-01 0.0000e+00 0.0000e+00 6.9336e-01
 6.9336e-01 1.4425e+02 6.9336e-01 0.0000e+00 6.9385e-01 6.2347e-02
 3.8891e-03 0.0000e+00 1.0986e+00 1.4438e+02 6.9336e-01 1.0986e+00
 3.8330e-01 3.4473e-01 1.1884e-01 0.0000e+00 6.9336e-01 7.9750e+01
 6.9336e-01 6.9336e-01 6.8994e-01 4.7974e-02 2.3003e-03 0.0000e+00
 6.9336e-01 1.4350e+02 6.9336e-01 6.9336e-01 3.9331e-01 3.4351e-01
 1.1804e-01 0.0000e+00 6.9336e-01 8.1812e+01 6.9336e-01 6.9336e-01]
8


In [66]:
file_name = './data.npz'
np.savez(file_name, features=features_array, labels=labels_array)

In [67]:
loaded_data = np.load(file_name)
features = loaded_data['features']
labels = loaded_data['labels']

In [68]:
features.shape

(278633, 72)

In [69]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(features, labels, shuffle=True, random_state=123,
                                                    stratify=labels)
print(X_train.shape)

(208974, 72)


In [70]:

from xgboost import XGBClassifier
from imblearn.over_sampling import ADASYN
from sklearn.metrics import accuracy_score, balanced_accuracy_score

adasyn = ADASYN(random_state=123)
X_train, Y_train = adasyn.fit_resample(X_train, Y_train)

In [71]:
X_train.shape

(677563, 72)

In [72]:
params_gpu = {
    'objective': 'binary:logistic',
    'tree_method': 'hist',  # Use histogram method
    'device': 'cuda',  # Set device to CUDA for GPU usage
    'eval_metric': 'logloss'
}

model = XGBClassifier(
    objective='multi:softmax',  # 다중 클래스 분류
    eval_metric='mlogloss',  # 다중 클래스 로지스틱 손실
    random_state=123,
    learning_rate=0.1,
    max_depth=6,
    n_estimators=100,
    device="cuda"
)
model.fit(X_train, Y_train, verbose=True)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device='cuda', early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.1, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=6,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=100,
              n_jobs=None, num_parallel_tree=None, objective='multi:softmax', ...)

In [74]:
import numpy as np
from bayes_opt import BayesianOptimization
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.metrics import make_scorer, balanced_accuracy_score


# 베이지안 최적화를 위한 함수 정의
def xgb_evaluate(max_depth, learning_rate, n_estimators, gamma, min_child_weight, subsample, colsample_bytree):
    # 모델 정의
    model = XGBClassifier(
        objective='multi:softmax',
        eval_metric='mlogloss',
        tree_method='hist',  # GPU 사용
        random_state=123,
        max_depth=int(max_depth),  # 정수형으로 변환
        learning_rate=learning_rate,
        n_estimators=int(n_estimators),  # 정수형으로 변환
        gamma=gamma,
        min_child_weight=min_child_weight,
        subsample=subsample,
        colsample_bytree=colsample_bytree,
        device='cuda'
    )

    # 교차 검증 설정
    cv = StratifiedKFold(n_splits=2, shuffle=True, random_state=123)

    # balanced accuracy scorer 정의
    balanced_acc_scorer = make_scorer(balanced_accuracy_score)

    # 교차 검증 점수 (balanced accuracy)를 계산
    scores = cross_val_score(model, X_train, Y_train, cv=cv, scoring=balanced_acc_scorer)
    return scores.mean()


# 파라미터 범위 설정
param_bounds = {
    'max_depth': (3, 10),
    'learning_rate': (0.01, 0.3),
    'n_estimators': (50, 500),
    'gamma': (0, 5),
    'min_child_weight': (1, 10),
    'subsample': (0.5, 1.0),
    'colsample_bytree': (0.5, 1.0)
}

# 베이지안 최적화 객체 생성
optimizer = BayesianOptimization(
    f=xgb_evaluate,
    pbounds=param_bounds,
    random_state=123,
    verbose=2
)

# 최적화 수행
optimizer.maximize(init_points=10, n_iter=30)

# 최적의 파라미터 출력
print("최적의 파라미터:", optimizer.max)


|   iter    |  target   | colsam... |   gamma   | learni... | max_depth | min_ch... | n_esti... | subsample |
-------------------------------------------------------------------------------------------------------------
| 1         | 0.8419    | 0.8482    | 1.431     | 0.07579   | 6.859     | 7.475     | 240.4     | 0.9904    |
| 2         | 0.8435    | 0.8424    | 2.405     | 0.1237    | 5.402     | 7.561     | 247.4     | 0.5298    |
| 3         | 0.7935    | 0.699     | 3.69      | 0.06292   | 4.228     | 5.784     | 289.3     | 0.8172    |
| 4         | 0.8746    | 0.9247    | 3.622     | 0.1872    | 8.057     | 3.907     | 212.8     | 0.6141    |
| 5         | 0.8148    | 0.6469    | 3.155     | 0.03671   | 6.036     | 4.878     | 272.2     | 0.7129    |
| 6         | 0.9035    | 0.6561    | 2.132     | 0.2691    | 9.609     | 5.517     | 330.8     | 0.5578    |
| 7         | 0.8627    | 0.6586    | 2.074     | 0.2612    | 4.753     | 5.347     | 493.5     | 0.7597    |
| 8       

In [75]:
def create_best_model(params):
    # 최적 파라미터를 정수형으로 변환
    max_depth = int(params['max_depth'])
    n_estimators = int(params['n_estimators'])

    # 최적 모델 정의
    best_model = XGBClassifier(
        objective='multi:softmax',
        eval_metric='mlogloss',
        tree_method='hist',  # GPU 사용
        device='cuda',  # GPU 설정
        random_state=123,
        max_depth=max_depth,
        learning_rate=params['learning_rate'],
        n_estimators=n_estimators,
        gamma=params['gamma'],
        min_child_weight=params['min_child_weight'],
        subsample=params['subsample'],
        colsample_bytree=params['colsample_bytree']
    )
    return best_model


best_params = optimizer.max['params']

# 최적의 파라미터를 적용하여 모델 생성
best_model = create_best_model(best_params)

# 모델 학습
best_model.fit(X_train, Y_train)

# 모델 평가 (테스트 데이터 사용)
Y_pred = best_model.predict(X_test)
balanced_acc = balanced_accuracy_score(Y_test, Y_pred)
print("Balanced Accuracy:", balanced_acc)

Balanced Accuracy: 0.8557413340522142
